<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS" data-toc-modified-id="JWST-Pipeline-Validation-Testing-Notebook:-MIRI-MRS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>JWST Pipeline Validation Testing Notebook: MIRI MRS</a></span><ul class="toc-item"><li><span><a href="#Spec2:-Assign_wcs()" data-toc-modified-id="Spec2:-Assign_wcs()-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Spec2: Assign_wcs()</a></span><ul class="toc-item"><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Table of Contents</a></span></li></ul></li></ul></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Calibration-WG-Requested-Algorithm:" data-toc-modified-id="Calibration-WG-Requested-Algorithm:-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Calibration WG Requested Algorithm:</a></span></li></ul></li></ul></li><li><span><a href="#Loading-Data" data-toc-modified-id="Loading-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Loading Data</a></span></li><li><span><a href="#Run-JWST-Pipeline---AssignWCS" data-toc-modified-id="Run-JWST-Pipeline---AssignWCS-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run JWST Pipeline - AssignWCS</a></span><ul class="toc-item"><li><span><a href="#Output-checks-and-tests" data-toc-modified-id="Output-checks-and-tests-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Output checks and tests</a></span></li><li><span><a href="#About-this-Notebook" data-toc-modified-id="About-this-Notebook-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>About this Notebook</a></span></li></ul></li></ul></div>

<a id="title_ID"></a>
# JWST Pipeline Validation Testing Notebook: MIRI MRS

## Spec2: Assign_wcs()
<span style="color:red"> **Instruments Affected**</span>: FGS, MIRI, NIRCam, NIRISS, NIRSpec 

### Table of Contents
<div style="text-align: left"> 

<br> [Imports](#imports_ID) <br> [Introduction](#intro_ID) <br> [Loading Data](#data_ID) <br> [Run JWST Pipeline](#pipeline_ID) <br> [Print Output](#residual_ID) <br> [About This Notebook](#about_ID) <br>

</div>

<a id="imports_ID"></a>
# Imports
List the library imports and why they are relevant to this notebook.

* os, glob for general  OS operations
* numpy
* astropy.io for opening fits files
* astropy.utils for remote data retrieval
* IPython.display for printing markdown output
* jwst.datamodels for building model for JWST Pipeline
* jwst.module.PipelineStep is the pipeline step being tested
* miricoord for coordinate conversion tools

[Top of Page](#title_ID)

In [1]:
import glob
import numpy as np
from numpy.testing import assert_allclose
import os

import astropy.io.fits as fits
import astropy.io.ascii as ascii
from ci_watson.artifactory_helpers import get_bigdata
from IPython.display import Markdown

from jwst.assign_wcs import AssignWcsStep
import miricoord.mrs.toolversions.mrs_pipetools_cdp8b as tv
from miricoord.mrs import mrs_tools as mrst

%matplotlib inline

<a id="intro_ID"></a>
# Introduction


In this notebook we will test the assign_wcs step of Spec2Pipeline() for **MRS** observations.

Step description: https://jwst-pipeline.readthedocs.io/en/latest/jwst/assign_wcs/index.html#assign-wcs-step

Pipeline code: https://github.com/spacetelescope/jwst/tree/master/jwst/assign_wcs


### Calibration WG Requested Algorithm: 

https://outerspace.stsci.edu/display/JWSTCC/Vanilla+Spectral+GWCS+Information



[Top of Page](#title_ID)

<a id="data_ID"></a>
# Loading Data

We are using simulated MRS observations, generated with MIRISim v2.3.0 (as of Dec 2020). ***(Needs updating)*** 

[Top of Page](#title_ID)

In [2]:
mock12A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12A-0.fits')

mock12B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12B-0.fits')

mock12C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock12C-0.fits')

mock34A = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34A-0.fits')

mock34B = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34B-0.fits')

mock34C = get_bigdata('jwst_validation_notebooks',
                 'validation_data',
                 'assign_wcs',
                 'assign_wcs_miri_test',
                 'mock34C-0.fits')

files = [mock12A, mock12B, mock12C, mock34A, mock34B, mock34C]

Collect configuration files locally if they aren't yet there.

In [3]:
if not os.path.exists('../cfg_files/'):
    os.mkdir('../cfg_files/')
    cfgs = collect_pipeline_cfgs.collect_pipeline_cfgs(dst='../cfg_files/')

<a id="pipeline_ID"></a>
# Run JWST Pipeline - AssignWCS

For this test we do not need to run the data through the Detector1() pipeline (in fact, this data will fail in Detector1). We begin by running the data through AssignWCS in spec2 using the 'assign_wcs.cfg' file. The AssignWcs step should not change anything in the science data or error extension. It attaches the WCS information to the data, so that a location in detector pixels can be reconstructed to an RA, dec, wavelength in spatial and spectral space.

[Top of Page](#title_ID)

In [4]:
awcs_out = []

for dd in files: #det1_out:
    awcs = AssignWcsStep.call(dd, config_file='../cfg_files/assign_wcs.cfg', save_results=True)
    awcs_out.append(awcs)

2021-04-05 18:03:04,635 - stpipe.assign_wcs - INFO - AssignWcsStep instance created.
2021-04-05 18:03:04,838 - stpipe.assign_wcs - INFO - Step assign_wcs running with args ('/Users/mengesser/Documents/jwst_validation_notebooks/jwst_validation_notebooks/assign_wcs/mock12A-0.fits',).
2021-04-05 18:03:04,840 - stpipe.assign_wcs - INFO - Step assign_wcs parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2021-04-05 18:03:34,138 - stpipe.assign_wcs - WARNING - /Users/mengesser/miniconda3/envs/jwpipe75/lib/python3.7/site-packages/asdf/asdf.py:208: UserWarning: File 'file:///grp/crds/cache/references/jwst/jwst_miri_wavelengthrange_0005.asdf' was created with extension 'asdf.extension.BuiltinExtension' from package asdf-2.7.1, but older vers

2021-04-05 18:05:17,823 - stpipe.assign_wcs - INFO - Created a MIRI mir_mrs pipeline with references {'distortion': '/grp/crds/cache/references/jwst/jwst_miri_distortion_0029.asdf', 'filteroffset': None, 'specwcs': '/grp/crds/cache/references/jwst/jwst_miri_specwcs_0022.asdf', 'regions': '/grp/crds/cache/references/jwst/jwst_miri_regions_0028.asdf', 'wavelengthrange': '/grp/crds/cache/references/jwst/jwst_miri_wavelengthrange_0005.asdf', 'camera': None, 'collimator': None, 'disperser': None, 'fore': None, 'fpa': None, 'msa': None, 'ote': None, 'ifupost': None, 'ifufore': None, 'ifuslicer': None}
2021-04-05 18:05:18,746 - stpipe.assign_wcs - WARNING - /Users/mengesser/miniconda3/envs/jwpipe75/lib/python3.7/site-packages/gwcs/wcs.py:131: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating t

2021-04-05 18:06:38,997 - stpipe.assign_wcs - INFO - COMPLETED assign_wcs
2021-04-05 18:06:39,437 - stpipe.assign_wcs - WARNING - /Users/mengesser/miniconda3/envs/jwpipe75/lib/python3.7/site-packages/gwcs/wcs.py:131: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  transforms = np.array(self._pipeline[from_ind: to_ind])[:, 1].copy()

2021-04-05 18:06:40,597 - stpipe.assign_wcs - INFO - Saved model in mock34B-0_assign_wcs.fits
2021-04-05 18:06:40,598 - stpipe.assign_wcs - INFO - Step assign_wcs done
2021-04-05 18:06:43,196 - stpipe.assign_wcs - INFO - AssignWcsStep instance created.
2021-04-05 18:06:43,614 - stpipe.assign_wcs - INFO - Step assign_wcs running with args ('/Users/mengesser/Documents/jwst_validation_notebooks/jwst_validation_notebooks/assign_wcs/mock34C-0.fits

<a id="residual_ID"></a>
## Output checks and tests

We can use the output model to run the following checks:

* the SCI extension is unchanged from the step input (accept default tolerances for the np.allclose() function)
* we use the wcs information assigned in the step to check the RA, dec, wavelength assignment at some reference locations. 


[Top of Page](#title_ID)

In [5]:
# Do a numeric check between input and output
for (aa, dd) in zip(awcs_out, files):
    data = fits.getdata(dd)
    filename = os.path.split(dd)[-1]
    assert np.allclose(aa.data, data, equal_nan=True), "Step input and output don't match for {}".format(filename)

Load the set of test coordinates from ``miricoord``. Then calculate the wavelength of these coordinates from the assign_wcs output models, and verify that they are close. If all the ``assert`` tests pass, this test is successful and complete.

In [6]:
def verify_coords(imgnum, ch, ref_data):
    """Uses predefined test cooridnates to check WCS output."""
    
    xt = ref_data[ch]['x']
    yt = ref_data[ch]['y']
    v2t = ref_data[ch]['v2']
    v3t = ref_data[ch]['v3']
    lamt = ref_data[ch]['lam']
    
    lamtest1 = np.zeros(2)
    lamtest2 = np.zeros(2)
    atest1 = np.zeros(2)
    btest1 = np.zeros(2)
    v2test1 = np.zeros(2)
    v3test1 = np.zeros(2)
    lt1 = np.zeros(2)
    
    print('**** TEST SET {}****'.format(ch))
    print(' x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test')
    for i in range (2):
        values = mrst.xytoabl(xt[i], yt[i], ch)
        atest1[i], btest1[i], lt1[i] = values['alpha'], values['beta'], values['lam']
        v2test1[i], v3test1[i] = mrst.abtov2v3(atest1[i], btest1[i], ch)
        ra, dec, lamtest1[i] = awcs_out[imgnum].meta.wcs(xt[i], yt[i])
        print('{0}  {1}   {2:.5f}    {3:.5f}    {4:.5f}   {5:.5f}   {6:.5f}   {7:.5f}'.format(
            xt[i], yt[i], lamt[i], lamtest1[i], v2t[i], v3t[i], v2test1[i], v3test1[i]))

        assert_allclose(lamt[i], lamtest1[i], atol=0.05)
        assert_allclose(v2t[i], v2test1[i], atol=0.05)
        assert_allclose(v3t[i], v3test1[i], atol=0.05)
        
    print('\n')

In [7]:
mrd = tv.mrs_ref_data
chan = {'0':'1A',
        '1':'1B',
        '2':'1C',
        '3':'2A',
        '4':'2B',
        '5':'2C',
        '6':'3A',
        '7':'3B',
        '8':'3C',
        '9':'4A',
        '10':'4B',
        '11':'4C'}

for i in range(12):
    ch = chan[str(i)]
    
    if i < 6 and 'A' in ch:
        imgnum = 0
    elif i < 6 and 'B' in ch:
        imgnum = 1
    elif i < 6 and 'C' in ch:
        imgnum = 2
    elif i >= 6 and 'A' in ch:
        imgnum = 3
    elif i >= 6 and 'B' in ch:
        imgnum = 4
    elif i >= 6 and 'C' in ch:
        imgnum = 5
        
    verify_coords(imgnum, ch, mrd)

**** TEST SET 1A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
76.0  512.0   5.34855    5.34855    -503.57285   -318.57499   -503.57285   -318.57499
354.0  700.0   5.51364    5.51364    -503.49798   -317.50901   -503.49798   -317.50901


**** TEST SET 1B****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
76.0  512.0   6.16831    6.16831    -503.64310   -318.72773   -503.64310   -318.72773
355.0  700.0   6.35801    6.35801    -503.37070   -317.69382   -503.37070   -317.69382


**** TEST SET 1C****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
78.0  512.0   7.00661    7.00661    -503.55984   -318.43677   -503.55984   -318.43677
356.0  700.0   7.21846    7.21846    -503.45976   -317.37795   -503.45976   -317.37795


**** TEST SET 2A****
 x       y     lam_truth  lamtest1   v2truth   v3truth  v2test   v3test
574.0  512.0   8.13946    8.13946    -503.65782   -319.37098   -503.65782   -319.37098
719.0  700.0   8

<a id="about_ID"></a>
## About this Notebook
**Author:** Mike Engesser, Science Support Analyst, MIRI branch 
<br>(Adapted from Sarah Kendrew's AssignWcs LRS-SLIT notebook.)
<br>**Updated On:** 4/01/2021

[Top of Page](#title_ID)
<img style="float: right;" src="./stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="stsci_pri_combo_mark_horizonal_white_bkgd" width="200px"/> 